In [21]:
!pip install pandas numpy tldextract scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import re
import tldextract
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import  confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import  RandomizedSearchCV

In [23]:
# loads csv files into legitmate and phishing dataframes
url_1 = pd.read_csv('Phishing URLs.csv')

url_2 = pd.read_csv('URL dataset.csv')

# Combine both DataFrames by rows (stacking them)
url_df = pd.concat([url_1, url_2], ignore_index=True)
print(url_df.head())

                                                 url      Type
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing
2                        https://kq0hgp.webwave.dev/  Phishing
3  https://brittishtele1bt-69836.getresponsesite....  Phishing
4         https://bt-internet-105056.weeblysite.com/  Phishing


In [24]:
# Calculates the length of each URL
url_df['URL_len'] = url_df['url'].str.len()

print(url_df.head())

                                                 url      Type  URL_len
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47
2                        https://kq0hgp.webwave.dev/  Phishing       27
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50
4         https://bt-internet-105056.weeblysite.com/  Phishing       42


In [25]:
# Calculates the length of hostname
url_df['Hostname_len'] = url_df['url'].apply(lambda x: len(urlparse(x).netloc))

print(url_df.head())


                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len  
0            15  
1            38  
2            18  
3            41  
4            33  


In [26]:
# Finds the number of subdomains
url_df['Hostname'] = url_df['url'].apply(lambda x: urlparse(x).netloc)

print(url_df.head())

url_df['num_subdomain'] = url_df['Hostname'].apply(lambda x: x.count('.')-1)  # -1 to not include the '.' in the main domain
print(url_df.head())


                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  
0            15                            docs.google.com  
1            38     btttelecommunniccatiion.weeblysite.com  
2            18                         kq0hgp.webwave.dev  
3            41  brittishtele1bt-69836.getresponsesite.com  
4            33          bt-internet-105056.weeblysite.com  
                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccati

In [27]:
#Number of special characters

def extract_alpha(string):
    alpha = []

    for char in string:
        if char.isalpha():
            alpha.append(char)
    return len(alpha)


def extract_nums(string):
    nums = []

    for char in string:
        if char.isnumeric():
            nums.append(char)

    return len(nums)

url_df['num_special_char'] = url_df['URL_len']  - url_df['url'].apply(lambda x: extract_alpha(x) + extract_nums(x)) 

print(url_df.head())

                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  num_subdomain  \
0            15                            docs.google.com              1   
1            38     btttelecommunniccatiion.weeblysite.com              1   
2            18                         kq0hgp.webwave.dev              1   
3            41  brittishtele1bt-69836.getresponsesite.com              1   
4            33          bt-internet-105056.weeblysite.com              1   

   num_special_char  
0                23  
1                 6  
2                 6 

In [28]:
#Number of digits
def extract_nums(string):
    nums = []

    for char in string:
        if char.isnumeric():
            nums.append(char)

    return len(nums)

url_df['num_digit'] = url_df['url'].apply(lambda x:extract_nums(x)) 

print(url_df.head())


                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  num_subdomain  \
0            15                            docs.google.com              1   
1            38     btttelecommunniccatiion.weeblysite.com              1   
2            18                         kq0hgp.webwave.dev              1   
3            41  brittishtele1bt-69836.getresponsesite.com              1   
4            33          bt-internet-105056.weeblysite.com              1   

   num_special_char  num_digit  
0                23         20  
1                 6 

In [29]:
#Check for https
url_df['https'] = url_df['url'].apply(lambda x: 1 if urlparse(x).scheme == 'https' else 0)

print(url_df.head())

                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  num_subdomain  \
0            15                            docs.google.com              1   
1            38     btttelecommunniccatiion.weeblysite.com              1   
2            18                         kq0hgp.webwave.dev              1   
3            41  brittishtele1bt-69836.getresponsesite.com              1   
4            33          bt-internet-105056.weeblysite.com              1   

   num_special_char  num_digit  https  
0                23         20      1  
1     

In [30]:
#Check for url with IP address
ipv4_pattern = r'^\d{1,3}(\.\d{1,3}){3}$'
ipv6_pattern = r'^[0-9a-fA-F:]+$'

def check_ip(string):
    hostname = urlparse(string).netloc

    if (re.match(ipv4_pattern, hostname) or (re.match(ipv6_pattern, hostname)) ):
        return 1
    else:
        return 0

url_df['ip_address'] = url_df['url'].apply(lambda x: check_ip(x))

print(url_df.head())



                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  num_subdomain  \
0            15                            docs.google.com              1   
1            38     btttelecommunniccatiion.weeblysite.com              1   
2            18                         kq0hgp.webwave.dev              1   
3            41  brittishtele1bt-69836.getresponsesite.com              1   
4            33          bt-internet-105056.weeblysite.com              1   

   num_special_char  num_digit  https  ip_address  
0                23         20    

In [31]:
# obtain tld
def check_tld(string):
    extracted_tld = tldextract.extract(string).suffix
    
    return extracted_tld

url_df['tld'] = url_df['url'].apply(lambda x: check_tld(x))
print(url_df.head())


                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  num_subdomain  \
0            15                            docs.google.com              1   
1            38     btttelecommunniccatiion.weeblysite.com              1   
2            18                         kq0hgp.webwave.dev              1   
3            41  brittishtele1bt-69836.getresponsesite.com              1   
4            33          bt-internet-105056.weeblysite.com              1   

   num_special_char  num_digit  https  ip_address  tld  
0                23         2

In [32]:
#Common tld
def check_legit_tld(string):
    extracted_tld = tldextract.extract(string).suffix
    
    legit_tlds = ['com', 'org', 'edu', 'gov', 'net', 'int', 'mil', 'uk', 'dev']

    if extracted_tld in legit_tlds:
        return 1
    else:
        return 0
    

url_df['common_tld'] = url_df['url'].apply(lambda x: check_legit_tld(x))
print(url_df.head())



                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  num_subdomain  \
0            15                            docs.google.com              1   
1            38     btttelecommunniccatiion.weeblysite.com              1   
2            18                         kq0hgp.webwave.dev              1   
3            41  brittishtele1bt-69836.getresponsesite.com              1   
4            33          bt-internet-105056.weeblysite.com              1   

   num_special_char  num_digit  https  ip_address  tld  common_tld  
0                

In [33]:
# Applying one hot encoding - legitmate url (1) and phishing (0)
def check_url_real_or_fake(string):
    if (string == "legitimate"):
        return 1
    else: 
        return 0


url_df['legit_url'] = url_df['Type'].apply(lambda x: check_url_real_or_fake(x))
print(url_df.head())

# Seperate input and output
y = url_df['legit_url']
print(y)
url_df = url_df.drop('legit_url', axis=1)
print(url_df.head())

                                                 url      Type  URL_len  \
0  https://docs.google.com/presentation/d/e/2PACX...  Phishing      178   
1    https://btttelecommunniccatiion.weeblysite.com/  Phishing       47   
2                        https://kq0hgp.webwave.dev/  Phishing       27   
3  https://brittishtele1bt-69836.getresponsesite....  Phishing       50   
4         https://bt-internet-105056.weeblysite.com/  Phishing       42   

   Hostname_len                                   Hostname  num_subdomain  \
0            15                            docs.google.com              1   
1            38     btttelecommunniccatiion.weeblysite.com              1   
2            18                         kq0hgp.webwave.dev              1   
3            41  brittishtele1bt-69836.getresponsesite.com              1   
4            33          bt-internet-105056.weeblysite.com              1   

   num_special_char  num_digit  https  ip_address  tld  common_tld  legit_url  
0     

In [34]:
#Maybe remove unwanted columns
url_df = url_df.drop('tld', axis=1)
url_df = url_df.drop('url', axis=1)
url_df = url_df.drop('Type', axis=1)
url_df = url_df.drop('Hostname', axis=1)
print(url_df.size)
url_df.to_csv("processed_dataset.csv", index=False)

4039864


In [35]:
x_train, x_test, y_train_labels, y_test_labels = train_test_split(url_df, y, test_size=0.6, random_state=42)
x_val, x_test, y_val_labels, y_test_labels = train_test_split(x_test,y_test_labels, test_size=0.5, random_state=42)

print(x_train.head())
print(x_test.head())
print(y_train_labels.head())
print(y_test_labels.head())

print(x_val.head())
print(x_test.head())
print(y_val_labels.head())
print(y_test_labels.head())

print(x_train.size)


        URL_len  Hostname_len  num_subdomain  num_special_char  num_digit  \
172743       52            26              2                 7          1   
225865       31            14              1                 7          0   
316939       72            17              1                13          0   
197159       57            16              1                12         10   
286257       96            14              1                15          9   

        https  ip_address  common_tld  
172743      1           0           1  
225865      1           0           1  
316939      1           0           1  
197159      1           0           1  
286257      1           0           1  
        URL_len  Hostname_len  num_subdomain  num_special_char  num_digit  \
422094       62            14              0                13         14   
284577       49            21              1                 8          8   
400482       29            20              1                 6    

In [ ]:
 #Hyperparameter tuning ()
rf = RandomForestClassifier(random_state= 42)

param_grid = {'n_estimators': [175, 200, 500],
              'max_depth': [None, 20, 30],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],
              "max_features": ["sqrt", "log2", None], 
              "bootstrap": [True],                     
              "class_weight": ["balanced", None]  }    

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=20,            
    cv=3,                 
    verbose=2,
    scoring="accuracy",
    random_state=42
)




# Fit the random search object to the data
random_search.fit(x_val, y_val_labels)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END bootstrap=True, class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=175; total time=  14.2s
[CV] END bootstrap=True, class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=175; total time=  12.9s
[CV] END bootstrap=True, class_weight=None, max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=175; total time=  13.3s
[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  14.4s
[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  14.9s
[CV] END bootstrap=True, class_weight=None, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  15.6s


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20,
                   param_distributions={'bootstrap': [True],
                                        'class_weight': ['balanced', None],
                                        'max_depth': [None, 20, 30],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [175, 200, 500]},
                   random_state=42, scoring='accuracy', verbose=2)

In [37]:
# Create a variable for the best model
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters for RF:',  random_search.best_params_)

Best hyperparameters for RF: {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 30, 'class_weight': None, 'bootstrap': True}


In [38]:
rf = RandomForestClassifier(max_depth= 30, n_estimators= 200,  min_samples_split= 10,min_samples_leaf = 2,max_features = 'log2',bootstrap = True, random_state= 42, n_jobs=-1)

rf.fit(x_train, y_train_labels)

y_pred = rf.predict(x_test)

accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred)
recall = recall_score(y_test_labels, y_pred)
f1 = f1_score(y_test_labels, y_pred)
print("Accuracy:", round(accuracy,2))
print("Precision:", round(precision,2))
print("Recall:", round(recall,2))
print("F1 Score:", round(f1,2))

#Confusion matrix
con_matrix = confusion_matrix(y_test_labels, y_pred) 
print(con_matrix)

Accuracy: 0.97
Precision: 0.97
Recall: 0.99
F1 Score: 0.98
[[ 44661   3440]
 [  1196 102198]]
